In [ ]:
# import packages
import requests
import pandas as pd


## Création d'une fonction pour récupérer les jeux de données et créer un dataframe

In [41]:
def get_indicator_data(url, columns=None):
    """
    Récupère et filtre les données d'un indicateur de l'OMS depuis l'API.

    Paramètres :
    - url (str) : URL de l'indicateur dans l'API de l'OMS.
    - columns (list, optionnel) : Liste des colonnes à conserver. Si None, conserve toutes les colonnes.

    Retourne :
    - pd.DataFrame : DataFrame avec les données récupérées et filtrées.
      Retourne None si la requête échoue.
    """
    # Effectuer la requête vers l'API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Charger les données JSON dans un DataFrame
        data = response.json()
        df = pd.DataFrame(data['value'])
        
        # Filtrer les colonnes si une liste est fournie
        if columns:
            existing_columns = [col for col in columns if col in df.columns]
            df_filtered = df[existing_columns]
        else:
            # Conserver toutes les colonnes si 'columns' est None
            df_filtered = df
        
        # Supprimer les lignes entièrement vides (optionnel)
        df_filtered = df_filtered.dropna(how='all')
        
        return df_filtered
    else:
        # Afficher un message d'erreur si la requête échoue
        print("Erreur lors de la récupération des données.")
        return None



In [42]:
# Exemple d'utilisation de la fonction pour obtenir toutes les colonnes de l'indicateur d'accessibilité des cigarettes
url_affordability = "https://ghoapi.azureedge.net/api/R_afford_gdp"
df_affordability_all_columns = get_indicator_data(url_affordability)
df_affordability_all_columns.head()




,Id,IndicatorCode,SpatialDimType,SpatialDim,ParentLocationCode,TimeDimType,ParentLocation,Dim1Type,TimeDim,Dim1,...,DataSourceDim,Value,NumericValue,Low,High,Comments,Date,TimeDimensionValue,TimeDimensionBegin,TimeDimensionEnd
0,349910,R_afford_gdp,COUNTRY,LSO,AFR,YEAR,Africa,None,2020,None,...,None,28.35,28.35,None,None,None,2023-12-15T14:03:42+01:00,2020,2020-01-01T00:00:00+01:00,2020-12-31T00:00:00+01:00
1,4093181,R_afford_gdp,COUNTRY,JOR,EMR,YEAR,Eastern Mediterranean,None,2014,None,...,None,4.09,4.09,None,None,None,2023-12-15T14:03:48+01:00,2014,2014-01-01T00:00:00+01:00,2014-12-31T00:00:00+01:00
2,6846398,R_afford_gdp,COUNTRY,GEO,EUR,YEAR,Europe,None,2016,None,...,None,2.6,2.60,None,None,None,2023-12-15T14:03:40+01:00,2016,2016-01-01T00:00:00+01:00,2016-12-31T00:00:00+01:00
3,824889,R_afford_gdp,COUNTRY,MHL,WPR,YEAR,Western Pacific,None,2014,None,...,None,6.32,6.32,None,None,None,2023-12-15T14:03:49+01:00,2014,2014-01-01T00:00:00+01:00,2014-12-31T00:00:00+01:00
4,6198817,R_afford_gdp,COUNTRY,LSO,AFR,YEAR,Africa,None,2012,None,...,None,29.82,29.82,None,None,None,2023-12-15T14:03:36+01:00,2012,2012-01-01T00:00:00+01:00,2012-12-31T00:00:00+01:00


In [43]:
# Exemple d'utilisation avec filtrage des colonnes spécifiées
columns_affordability = ['SpatialDim', 'TimeDim', 'Value', 'NumericValue', 'Low', 'High']
df_affordability_filtered = get_indicator_data(url_affordability, columns_affordability)
df_affordability_filtered.head()


,SpatialDim,TimeDim,Value,NumericValue,Low,High
0,LSO,2020,28.35,28.35,None,None
1,JOR,2014,4.09,4.09,None,None
2,GEO,2016,2.6,2.60,None,None
3,MHL,2014,6.32,6.32,None,None
4,LSO,2012,29.82,29.82,None,None
